In [ ]:
!pip install ultralytics
!pip install gradio
!pip install pillow
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [ ]:
import gradio as gr
import torch
from ultralytics import YOLO
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import json
import io
import base64

# Check if CUDA is available
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA device count: {torch.cuda.device_count()}")
if torch.cuda.is_available():
    print(f"Current CUDA device: {torch.cuda.current_device()}")
    print(f"Device name: {torch.cuda.get_device_name()}")

CUDA available: False
CUDA device count: 0


In [ ]:
class YOLODetector:
    def __init__(self, model_name="yolov8n.pt"):
        """
        Initialize YOLO detector
        model_name options:
        - yolov8n.pt (nano - fastest, least accurate)
        - yolov8s.pt (small)
        - yolov8m.pt (medium)
        - yolov8l.pt (large)
        - yolov8x.pt (extra large - slowest, most accurate)
        """
        print(f"Loading YOLO model: {model_name}")
        self.model = YOLO(model_name)

        # Move to GPU if available
        if torch.cuda.is_available():
            self.model.to('cuda')
            print("Model moved to GPU")
        else:
            print("Using CPU")

    def detect_objects(self, image, confidence_threshold=0.25, iou_threshold=0.45):
        """
        Detect objects in image
        """
        # Run inference
        results = self.model(image, conf=confidence_threshold, iou=iou_threshold)

        # Parse results
        detections = []
        annotated_image = image.copy()

        if results and len(results) > 0:
            result = results[0]

            if result.boxes is not None and len(result.boxes) > 0:
                # Get class names
                names = result.names

                # Draw on image
                draw = ImageDraw.Draw(annotated_image)

                # Try to load a font, fall back to default if not available
                try:
                    font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf", 16)
                except:
                    font = ImageFont.load_default()

                for box in result.boxes:
                    # Get box coordinates
                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                    confidence = box.conf[0].cpu().numpy()
                    class_id = int(box.cls[0].cpu().numpy())
                    class_name = names[class_id]

                    # Store detection info
                    detections.append({
                        'class': class_name,
                        'confidence': float(confidence),
                        'bbox': [float(x1), float(y1), float(x2), float(y2)]
                    })

                    # Draw bounding box
                    draw.rectangle([x1, y1, x2, y2], outline="red", width=3)

                    # Draw label
                    label = f"{class_name}: {confidence:.2f}"

                    # Get text size for background rectangle
                    bbox = draw.textbbox((x1, y1-25), label, font=font)
                    draw.rectangle([bbox[0]-2, bbox[1]-2, bbox[2]+2, bbox[3]+2], fill="red")
                    draw.text((x1, y1-25), label, fill="white", font=font)

        return detections, annotated_image

    def get_detection_summary(self, detections):
        """
        Create a summary of detected objects
        """
        if not detections:
            return "No objects detected in the image."

        # Count objects by class
        class_counts = {}
        for det in detections:
            class_name = det['class']
            class_counts[class_name] = class_counts.get(class_name, 0) + 1

        # Create summary
        summary_parts = []
        for class_name, count in class_counts.items():
            if count == 1:
                summary_parts.append(f"1 {class_name}")
            else:
                summary_parts.append(f"{count} {class_name}s")

        summary = "Detected: " + ", ".join(summary_parts)
        return summary

# Initialize the detector
detector = YOLODetector("yolov8n.pt")  # You can change this to yolov8s.pt, yolov8m.pt, etc.

Loading YOLO model: yolov8n.pt
Using CPU


In [ ]:
def analyze_image(image, confidence_threshold, iou_threshold):
    """
    Main function for the Gradio interface
    """
    if image is None:
        return None, "Please upload an image.", ""

    try:
        # Convert to PIL Image if needed
        if not isinstance(image, Image.Image):
            image = Image.fromarray(image)

        # Ensure RGB mode
        if image.mode != 'RGB':
            image = image.convert('RGB')

        # Detect objects
        detections, annotated_image = detector.detect_objects(
            image,
            confidence_threshold=confidence_threshold,
            iou_threshold=iou_threshold
        )

        # Create summary
        summary = detector.get_detection_summary(detections)

        # Create detailed results
        detailed_results = []
        for i, det in enumerate(detections, 1):
            detailed_results.append(
                f"{i}. {det['class'].title()}: {det['confidence']:.1%} confidence"
            )

        detailed_text = "\n".join(detailed_results) if detailed_results else "No objects detected."

        return annotated_image, summary, detailed_text

    except Exception as e:
        return None, f"Error processing image: {str(e)}", ""

def batch_analyze(files, confidence_threshold, iou_threshold):
    """
    Analyze multiple images at once
    """
    if not files:
        return "No files uploaded."

    results = []
    for i, file in enumerate(files):
        try:
            image = Image.open(file.name).convert('RGB')
            detections, _ = detector.detect_objects(
                image,
                confidence_threshold=confidence_threshold,
                iou_threshold=iou_threshold
            )
            summary = detector.get_detection_summary(detections)
            results.append(f"Image {i+1}: {summary}")
        except Exception as e:
            results.append(f"Image {i+1}: Error - {str(e)}")

    return "\n".join(results)

In [ ]:
# Create the main interface
with gr.Blocks(title="YOLO Object Detection API", theme=gr.themes.Soft()) as demo:
    gr.Markdown("""
    # 🎯 YOLO Object Detection API

    Upload an image and get real-time object detection results using YOLOv8!

    **Features:**
    - Real-time object detection
    - Confidence score adjustment
    - Batch processing
    - Visual annotations
    - JSON API endpoints
    """)

    with gr.Tab("Single Image Detection"):
        with gr.Row():
            with gr.Column(scale=1):
                input_image = gr.Image(
                    type="pil",
                    label="Upload Image",
                    height=400
                )

                with gr.Row():
                    confidence_slider = gr.Slider(
                        minimum=0.1,
                        maximum=1.0,
                        value=0.25,
                        step=0.05,
                        label="Confidence Threshold"
                    )
                    iou_slider = gr.Slider(
                        minimum=0.1,
                        maximum=1.0,
                        value=0.45,
                        step=0.05,
                        label="IoU Threshold"
                    )

                analyze_btn = gr.Button("🔍 Analyze Image", variant="primary")

            with gr.Column(scale=1):
                output_image = gr.Image(
                    label="Detection Results",
                    height=400
                )

                summary_text = gr.Textbox(
                    label="Quick Summary",
                    lines=2,
                    interactive=False
                )

                detailed_results = gr.Textbox(
                    label="Detailed Results",
                    lines=8,
                    interactive=False
                )

    with gr.Tab("Batch Processing"):
        with gr.Row():
            batch_files = gr.File(
                file_count="multiple",
                label="Upload Multiple Images",
                file_types=["image"]
            )

        with gr.Row():
            batch_confidence = gr.Slider(0.1, 1.0, 0.25, step=0.05, label="Confidence Threshold")
            batch_iou = gr.Slider(0.1, 1.0, 0.45, step=0.05, label="IoU Threshold")

        batch_btn = gr.Button("🔍 Analyze All Images", variant="primary")
        batch_results = gr.Textbox(
            label="Batch Results",
            lines=15,
            interactive=False
        )

    with gr.Tab("API Information"):
        gr.Markdown("""
        ## 📋 API Usage Information

        ### Model Details
        - **Model**: YOLOv8 Nano (fastest inference)
        - **Classes**: 80 COCO dataset classes
        - **Input**: Images (JPG, PNG, etc.)
        - **Output**: Bounding boxes, class names, confidence scores

        ### Available Classes
        The model can detect these objects:

        `person, bicycle, car, motorcycle, airplane, bus, train, truck, boat, traffic light,
        fire hydrant, stop sign, parking meter, bench, bird, cat, dog, horse, sheep, cow,
        elephant, bear, zebra, giraffe, backpack, umbrella, handbag, tie, suitcase, frisbee,
        skis, snowboard, sports ball, kite, baseball bat, baseball glove, skateboard, surfboard,
        tennis racket, bottle, wine glass, cup, fork, knife, spoon, bowl, banana, apple,
        sandwich, orange, broccoli, carrot, hot dog, pizza, donut, cake, chair, couch,
        potted plant, bed, dining table, toilet, tv, laptop, mouse, remote, keyboard,
        cell phone, microwave, oven, toaster, sink, refrigerator, book, clock, vase,
        scissors, teddy bear, hair drier, toothbrush`

        ### Parameters
        - **Confidence Threshold**: Minimum confidence score for detections (0.1-1.0)
        - **IoU Threshold**: Intersection over Union threshold for Non-Maximum Suppression (0.1-1.0)

        ### Performance Tips
        - Lower confidence threshold = more detections (including false positives)
        - Higher confidence threshold = fewer, more confident detections
        - Adjust IoU threshold to control overlapping detections
        """)

    # Connect the functions
    analyze_btn.click(
        fn=analyze_image,
        inputs=[input_image, confidence_slider, iou_slider],
        outputs=[output_image, summary_text, detailed_results]
    )

    batch_btn.click(
        fn=batch_analyze,
        inputs=[batch_files, batch_confidence, batch_iou],
        outputs=batch_results
    )

    # Example images
    gr.Examples(
        examples=[
            ["https://ultralytics.com/images/bus.jpg", 0.25, 0.45],
            ["https://ultralytics.com/images/zidane.jpg", 0.25, 0.45],
        ],
        inputs=[input_image, confidence_slider, iou_slider],
        outputs=[output_image, summary_text, detailed_results],
        fn=analyze_image,
        cache_examples=True,
    )

In [ ]:
# Launch the interface
if __name__ == "__main__":
    print("🚀 Starting YOLO Detection API...")
    print("📱 The interface will be available at the URL shown below")
    print("🌐 Share=True creates a public URL for external access")

    # Launch with public sharing enabled
    demo.launch(
        share=True,          # Creates public URL
        server_name="0.0.0.0",  # Allow external connections
        server_port=7860,    # Port number
        show_error=True,     # Show errors in interface
        quiet=False          # Show startup logs
    )

🚀 Starting YOLO Detection API...
📱 The interface will be available at the URL shown below
🌐 Share=True creates a public URL for external access
Using cache from '/content/.gradio/cached_examples/28' directory. If method or examples have changed since last caching, delete this folder to clear cache.

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b2dfae201b4dff6176.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
